In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
# 원본 데이터 read
customer_info = pd.read_csv('LPOINT_BIG_COMP_01_DEMO.csv') #고객정보
purchase_info = pd.read_csv('LPOINT_BIG_COMP_02_PDDE.csv') # 구매정보
product_info = pd.read_csv('LPOINT_BIG_COMP_04_PD_CLAC.csv') #제품 정보

In [3]:
 # 구매정보, 상품정보, 고객정보 join
purchase_joined = pd.merge(purchase_info,product_info)
purchase_joined = pd.merge(purchase_joined,customer_info)

## 온라인 구매내역 추출

In [4]:
#온라인구매내역 추출
purchase_online = purchase_joined[purchase_joined['chnl_dv']==2]

In [5]:
purchase_online

,cust,rct_no,chnl_dv,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct,pd_nm,clac_hlv_nm,clac_mcls_nm,ma_fem_dv,ages,zon_hlv
1941,M977958641,E02024894057,2,A02,NaN,PD1181,20210517,11,5940.0,3,커피음료,음료,커피음료,남성,30대,Z10
1986,M977958641,E02023587795,2,A02,NaN,PD0223,20210509,21,2480.0,1,젤리,과자,사탕/캔디,남성,30대,Z10
1987,M977958641,E02023587795,2,A02,NaN,PD0223,20210509,21,2480.0,1,젤리,과자,사탕/캔디,남성,30대,Z10
1988,M977958641,E02026074508,2,A02,NaN,PD0223,20210525,17,3480.0,1,젤리,과자,사탕/캔디,남성,30대,Z10
2003,M977958641,E02023587795,2,A02,NaN,PD0401,20210509,21,3480.0,1,즉석국/찌개,대용식,레토르트,남성,30대,Z10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4381738,M990831120,E06048952027,2,A06,NaN,PD0508,20211024,1,8500.0,1,영화/문화모바일상품권,상품권,모바일상품권,여성,30대,Z10
4381739,M990831120,E06049213300,2,A06,NaN,PD0508,20211026,12,17000.0,1,영화/문화모바일상품권,상품권,모바일상품권,여성,30대,Z10
4381740,M828090075,E06050917660,2,A06,NaN,PD0508,20211202,11,50000.0,1,영화/문화모바일상품권,상품권,모바일상품권,남성,30대,Z03
4381741,M317815845,E06042488774,2,A06,NaN,PD0510,20210623,16,50000.0,1,생활모바일상품권,상품권,모바일상품권,남성,40대,Z04


In [46]:
purchase_online.to_csv('purchase_online.csv',index=False)

## 카테고리 1개 구매 고객 csv만들기

In [50]:
# 구매카테고리가 한개인 고객 추출
on_custs_id = purchase_online['cust'].unique()
one_custs = []

for v in on_custs_id:
    if len(purchase_online[purchase_online['cust']==v].groupby('clac_hlv_nm').count().index)==1:
        one_custs.append(v)
one_custs_df = pd.DataFrame(one_custs)        

In [111]:
one_custs_df.to_csv('on_one_custs_df.csv',index=False)

## 카테고리 2개 이상 고객 추출

In [6]:
one_custs_df = pd.read_csv('on_one_custs_df.csv')

In [7]:
online_custs = set(purchase_online['cust'].to_list()) # 온라인 고객 id 리스트
one_custs = one_custs_df['0'].to_list() # 카테고리 1개 구매한 고객 id 리스트
over_two_custs = online_custs-set(one_custs) #카테고리 2개 이상 구매 고객 set

In [8]:
# 카테고리 2개 이상 구매한 고객들의 구매 내역 데이터프레임
custs_over_two_df = purchase_online[purchase_online['cust'].isin(over_two_custs)]

## 온라인 고객 카테고리별 구매금액 합 csv만들기

In [83]:
# 고객 구매 항목별 구매금액 df 
def get_buyam_per_prd(cid):
    temp = custs_over_two_df[custs_over_two_df['cust']==cid]
    return temp[['clac_hlv_nm','buy_am']].groupby('clac_hlv_nm').sum()

In [117]:
## 대분류 숫자 코드 dict 만들기
big_ct = custs_over_two_df['clac_hlv_nm'].unique()
big_ct_dict = {}
for i,v in enumerate(big_ct):
    big_ct_dict[v]=i

In [85]:
# 고객별 카테고리 구매금액 함 데이터프레임 만들기
over_two_custs_id = custs_over_two_df['cust'].unique()
init = np.zeros(shape=(len(big_ct),))
buyam_res = pd.DataFrame([init])

for i,v in enumerate(over_two_custs_id):
    temp = np.zeros(shape=(len(big_ct),))
    df = get_buyam_per_prd(v)
    for j,v2 in enumerate(df.index):
        temp[big_ct_dict[v2]] = df.loc[v2]['buy_am']
    buyam_res.loc[i] = temp

In [86]:
over_two_custs_id_df = pd.DataFrame(custs_over_two_df['cust'].unique())

In [88]:
online_buyam = pd.concat([buyam_res, over_two_custs_id_df ],axis=1)

In [89]:
col_nm = custs_over_two_df['clac_hlv_nm'].unique()
col_nm = np.append(col_nm, 'cust_id')
online_buyam.columns =col_nm

In [90]:
online_buyam_final = online_buyam[['cust_id','음료', '과자', '대용식', '침구/수예', '냉동식품', '조리식품', '채소', '유제품', '냉장식품', '축산물',
       '조미료', '퍼스널케어', '주방잡화', '화장품/뷰티케어', '속옷/양말/홈웨어', '건강식품', '건강용품',
       '세제/위생', '병통조림', '문구/사무용품', '패션잡화', '청소/세탁/욕실용품', '원예/애완', '수산물',
       '출산/육아용품', '모바일', '완구', '여성의류', '남성의류', '유아동의류', '스포츠패션', '커피/차',
       '여행/레저서비스', '생활/주방가전', '계절가전', '식기/조리기구', '아웃도어/레저', '상품권', '자동차용품',
       '헬스/피트니스', '컴퓨터', '양곡', '과일', '건해산물', '테넌트/음식점', '구기/필드스포츠', '가구',
       '인테리어/조명', '서적/음반/악기', '시즌스포츠', '유아식품', '공구/안전용품', '영상/음향가전', '기타(비상품)',
       '주류', '냉장/세탁가전', '생활/렌탈서비스']].copy()

In [91]:
online_buyam_final.to_csv('online_buyam.csv',index=False)

## 온라인 고객 카테고리별 구매개수 합 csv만들기

In [93]:
# 고객 구매 항목별 구매금액 df 
def get_cnt_per_prd(cid):
    temp = custs_over_two_df[custs_over_two_df['cust']==cid]
    return temp[['clac_hlv_nm','buy_ct']].groupby('clac_hlv_nm').sum()

In [94]:
## 대분류 숫자 코드 dict 만들기
big_ct = custs_over_two_df['clac_hlv_nm'].unique()
big_ct_dict = {}
for i,v in enumerate(big_ct):
    big_ct_dict[v]=i

In [95]:
over_two_custs_id = custs_over_two_df['cust'].unique()
init = np.zeros(shape=(len(big_ct),))
buycnt_res = pd.DataFrame([init])

for i,v in enumerate(over_two_custs_id):
    temp = np.zeros(shape=(len(big_ct),))
    df = get_cnt_per_prd(v)
    for j,v2 in enumerate(df.index):
        temp[big_ct_dict[v2]] = df.loc[v2]['buy_ct']
    buycnt_res.loc[i] = temp

In [ ]:
over_two_custs_id_df = pd.DataFrame(custs_over_two_df['cust'].unique())
online_buycnt = pd.concat([buycnt_res, over_two_custs_id_df ],axis=1)

In [ ]:
online_buycnt.columns =col_nm

In [ ]:
online_buycnt_final = online_buycnt[['cust_id','음료', '과자', '대용식', '침구/수예', '냉동식품', '조리식품', '채소', '유제품', '냉장식품', '축산물',
       '조미료', '퍼스널케어', '주방잡화', '화장품/뷰티케어', '속옷/양말/홈웨어', '건강식품', '건강용품',
       '세제/위생', '병통조림', '문구/사무용품', '패션잡화', '청소/세탁/욕실용품', '원예/애완', '수산물',
       '출산/육아용품', '모바일', '완구', '여성의류', '남성의류', '유아동의류', '스포츠패션', '커피/차',
       '여행/레저서비스', '생활/주방가전', '계절가전', '식기/조리기구', '아웃도어/레저', '상품권', '자동차용품',
       '헬스/피트니스', '컴퓨터', '양곡', '과일', '건해산물', '테넌트/음식점', '구기/필드스포츠', '가구',
       '인테리어/조명', '서적/음반/악기', '시즌스포츠', '유아식품', '공구/안전용품', '영상/음향가전', '기타(비상품)',
       '주류', '냉장/세탁가전', '생활/렌탈서비스']].copy()

In [ ]:
online_buycnt_final.to_csv('online_buycnt.csv',index=False)

## 온라인 고객 카테고리별 구매금액 합 데이터 전처리

In [9]:
# 온라인 고객 1년 구매금액 합(카테고리 2개 이상)
on_buyam = pd.read_csv('online_buyam.csv')

### normal 고객 카테고리별 구매금액 합 데이터프레임

In [10]:
# 200만원 이하 구매고객 
on_buy_am_sum = purchase_online[['cust','buy_am']].groupby('cust').sum()
on_normal = on_buy_am_sum[on_buy_am_sum['buy_am']<=2000000] #금액으로만 구분
on_normal_ids = on_normal.index.tolist()

#20만원 이상, 200만원 이하, 카테고리2개 이상 구매고객 구매금액 합 데이터
on_buyam_normal = on_buyam[on_buyam['cust_id'].isin(on_normal_ids)].reset_index(drop=True)

### 고객 인구통계학적 정보 합치기

In [11]:
 # 20만원 이상, 200만원 이하, 카테고리 2개 이상 구매 고객 id 데이터 프레임
on_final_normal_ids = on_buyam_normal[['cust_id']].rename(columns={'cust_id':'cust'})

In [12]:
on_normal_demo =  pd.merge(on_final_normal_ids,customer_info).iloc[:,:-1]

In [13]:
gender = pd.get_dummies(on_normal_demo['ma_fem_dv'],drop_first=True)
ages = pd.get_dummies(on_normal_demo['ages'])
on_normal_demo_oh= pd.concat([gender,ages],axis=1)

In [14]:
on_buyam_normal_d = pd.concat([on_buyam_normal,on_normal_demo_oh],axis=1)

### 구매금액 합 클러스터링

In [15]:
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans

In [16]:
# normalizer를 이용해서 스케일링하기
on_buyam_normal_pd = on_buyam_normal_d.drop(['cust_id'], axis=1).copy() # scaling을 위해 cust_id 드롭
col_nm = on_buyam_normal_pd.columns

nom = Normalizer(copy=False)
on_buyam_normal_pd_n = pd.DataFrame(nom.fit_transform(on_buyam_normal_pd))  # normalizer로 스케일링
on_buyam_normal_pd_n.columns = col_nm

In [17]:
kmeans = KMeans(n_clusters = 6, random_state=0)
am_clusters = kmeans.fit(on_buyam_normal_pd_n)

In [18]:
# 구매금액 합 최종
on_buyam_normal_final = on_buyam_normal_d.copy()
on_buyam_normal_final['cluster'] = am_clusters.labels_

# 구매금액(normalized) 합 최종
cid = on_buyam_normal_final[['cust_id']]
cluster = on_buyam_normal_final[['cluster']]
on_buyam_normal_n_final= pd.concat([cid,on_buyam_normal_pd_n],axis=1)
on_buyam_normal_n_final = pd.concat([on_buyam_normal_n_final,cluster],axis=1)

In [19]:
on_buyam_normal_final

,cust_id,음료,과자,대용식,침구/수예,냉동식품,조리식품,채소,유제품,냉장식품,...,냉장/세탁가전,생활/렌탈서비스,여성,20대,30대,40대,50대,60대,70대,cluster
0,M977958641,23120.0,24420.0,27730.0,56700.0,13460.0,7320.0,5860.0,14290.0,36880.0,...,0.0,0.0,0,0,1,0,0,0,0,4
1,M542006779,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,1,0,0,0,0,0,4
2,M081569058,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,1,0,0,0,4
3,M985499664,0.0,0.0,0.0,45900.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,1,0,0,0,2
4,M783908672,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,1,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5680,M281534825,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,1,0,0,0,0,3
5681,M184214376,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,1,0,0,0,4
5682,M851340618,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,1,0,0,0,4
5683,M951159146,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,1,0,0,0,0


In [20]:
on_buyam_normal_n_final

,cust_id,음료,과자,대용식,침구/수예,냉동식품,조리식품,채소,유제품,냉장식품,...,냉장/세탁가전,생활/렌탈서비스,여성,20대,30대,40대,50대,60대,70대,cluster
0,M977958641,0.130044,0.137356,0.155974,0.318922,0.075709,0.041173,0.032961,0.080377,0.20744,...,0.0,0.0,0.000000,0.000000,0.000006,0.000000,0.0,0.0,0.0,4
1,M542006779,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.000000,0.000003,0.000000,0.000000,0.0,0.0,0.0,4
2,M081569058,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.000009,0.000000,0.000000,0.000009,0.0,0.0,0.0,4
3,M985499664,0.000000,0.000000,0.000000,0.073058,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.000002,0.000000,0.000000,0.000002,0.0,0.0,0.0,2
4,M783908672,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.000002,0.000000,0.000000,0.000002,0.0,0.0,0.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5680,M281534825,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.000005,0.000000,0.000005,0.000000,0.0,0.0,0.0,3
5681,M184214376,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.000021,0.000000,0.000000,0.000021,0.0,0.0,0.0,4
5682,M851340618,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.000009,0.000000,0.000000,0.000009,0.0,0.0,0.0,4
5683,M951159146,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000012,0.0,0.0,0.0,0


In [21]:
on_buyam_normal_final.to_csv('on_buyam_normal_final.csv',index=False)

## 온라인 고객 카테고리별 구매개수 합 데이터 전처리

In [22]:
# 온라인 고객 1년 구매금액 합(카테고리 2개 이상)
on_buycnt = pd.read_csv('online_buycnt.csv')

### normal 고객 카테고리별 구매금액 합 데이터프레임

In [23]:
#20만원 이상, 200만원 이하, 카테고리2개 이상 구매고객 구매개수 합 데이터
on_buycnt_normal = on_buycnt[on_buycnt['cust_id'].isin(on_normal_ids)].reset_index(drop=True)

### 고객 인구통계학적 정보 합치기

In [24]:
on_buycnt_normal_d = pd.concat([on_buycnt_normal,on_normal_demo_oh],axis=1)

### 구매개수 클러스터링

In [25]:
# 스케일링
on_buycnt_normal_pd = on_buycnt_normal_d.drop(['cust_id'], axis=1).copy() # scaling을 위해 cust_id 드롭
col_nm = on_buycnt_normal_pd.columns

nom = Normalizer(copy=False)
on_buycnt_normal_pd_n = pd.DataFrame(nom.fit_transform(on_buycnt_normal_pd))  # normalizer로 스케일링
on_buycnt_normal_pd_n.columns = col_nm

In [26]:
kmeans = KMeans(n_clusters = 6, random_state=0)
cnt_clusters = kmeans.fit(on_buycnt_normal_pd_n)

In [27]:
# 구매금액 합 최종
on_buycnt_normal_final = on_buycnt_normal_d.copy()
on_buycnt_normal_final['cluster'] = cnt_clusters.labels_

# 구매금액(normalized) 합 최종
cid = on_buycnt_normal_final[['cust_id']]
cluster = on_buycnt_normal_final[['cluster']]
on_buycnt_normal_n_final= pd.concat([cid,on_buycnt_normal_pd_n],axis=1)
on_buycnt_normal_n_final = pd.concat([on_buycnt_normal_n_final,cluster],axis=1)

In [28]:
on_buycnt_normal_final

,cust_id,음료,과자,대용식,침구/수예,냉동식품,조리식품,채소,유제품,냉장식품,...,냉장/세탁가전,생활/렌탈서비스,여성,20대,30대,40대,50대,60대,70대,cluster
0,M977958641,9.0,11.0,7.0,4.0,2.0,3.0,3.0,5.0,6.0,...,0.0,0.0,0,0,1,0,0,0,0,0
1,M542006779,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,1,0,0,0,0,0,2
2,M081569058,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,1,0,0,0,4
3,M985499664,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,1,0,0,0,2
4,M783908672,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,1,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5680,M281534825,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,1,0,0,0,0,2
5681,M184214376,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,1,0,0,0,2
5682,M851340618,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,1,0,0,0,2
5683,M951159146,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,1,0,0,0,5


In [29]:
on_buycnt_normal_n_final

,cust_id,음료,과자,대용식,침구/수예,냉동식품,조리식품,채소,유제품,냉장식품,...,냉장/세탁가전,생활/렌탈서비스,여성,20대,30대,40대,50대,60대,70대,cluster
0,M977958641,0.305129,0.372935,0.237322,0.135613,0.067806,0.10171,0.10171,0.169516,0.203419,...,0.0,0.0,0.000000,0.000000,0.033903,0.000000,0.0,0.0,0.0,0
1,M542006779,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,...,0.0,0.0,0.000000,0.258199,0.000000,0.000000,0.0,0.0,0.0,2
2,M081569058,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,...,0.0,0.0,0.288675,0.000000,0.000000,0.288675,0.0,0.0,0.0,4
3,M985499664,0.000000,0.000000,0.000000,0.063888,0.000000,0.00000,0.00000,0.000000,0.000000,...,0.0,0.0,0.063888,0.000000,0.000000,0.063888,0.0,0.0,0.0,2
4,M783908672,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,...,0.0,0.0,0.115470,0.000000,0.000000,0.115470,0.0,0.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5680,M281534825,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,...,0.0,0.0,0.500000,0.000000,0.500000,0.000000,0.0,0.0,0.0,2
5681,M184214376,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,...,0.0,0.0,0.098533,0.000000,0.000000,0.098533,0.0,0.0,0.0,2
5682,M851340618,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,...,0.0,0.0,0.500000,0.000000,0.000000,0.500000,0.0,0.0,0.0,2
5683,M951159146,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.577350,0.0,0.0,0.0,5


In [30]:
on_buycnt_normal_final.to_csv('on_buycnt_normal_final.csv',index=False)